# データベースをパース 

In [1]:
from pymongo import MongoClient
import datetime
import pprint
import re
from pathlib import Path
import pprint

In [2]:
from pdf_parser_ver2 import PdfParser, DirectoryPdfParser, PdfParserCount

## データベースの設定

In [3]:
client = MongoClient('localhost', 27017)
db = client["db"]
collection = db["papers"]

## データベースから正規表現で検索

In [4]:
count_patterns = [re.compile("ディープラーニング|深層学習"),
                  re.compile("CNN|ニューラルネットワーク"),
                  re.compile("VAE|変分オートエンコーダ"),
                  re.compile("GAN"),
                 ]

## 複数の正規表現で検索

In [8]:
find_result = collection.find({"$or":[{"content.序論":re.compile("VAE|変分オートエンコーダ")},
                                      {"content.序論":re.compile("GAN")}]})

counter = 0
for i in find_result:
    counter += 1
    #pprint.pprint(i)
print(counter)

12


### パターンのリストで指定

In [11]:
content_name = "序論"
find_result = collection.find(filter={"$or":[{"content."+content_name:i} for i in count_patterns]}).sort("date")  # dateはないけど
counter = 0
for i in find_result:
    counter += 1
    #pprint.pprint(i)
print(counter)

62


## データベースから取得し，検索するクラス

In [13]:
class PaperDataBaseLoadPattern():
        def __init__(self, collection, find_patterns, content_name, find_max=100):
            self.collection = collection
            self.find_patterns = find_patterns
            self.content_name = content_name
            self.find_max = find_max
        def load(self):
            """
            検索結果からfind_max分だけ検索．呼んだ回数だけ検索結果が進む
            """
            result = collection.find(filter={"$or":[{"content."+self.content_name:i} for i in self.find_patterns]}).sort("date")  # dateはないけど
            out_list = []
            iter_counter = 0
            for paper in result:
                out_list.append(paper)
                iter_counter += 1
                if iter_counter >= self.find_max-1:
                    yield out_list
                    out_list = []  # リストの初期化
                    iter_counter = 0
            yield out_list #全て終わったときに返す

テストコード

In [14]:
find_patterns = [re.compile("ディープラーニング|深層学習"),
                 re.compile("CNN|ニューラルネットワーク"),
                 re.compile("VAE|変分オートエンコーダ"),
                 re.compile("GAN"),
                ]

In [16]:
database_loader = PaperDataBaseLoadPattern(collection=collection,
                                           content_name="序論",
                                           find_patterns=find_patterns,
                                           find_max=10
                                           )
database_loader_generator = database_loader.load()

In [18]:
dict_list = next(database_loader_generator)
pprint.pprint(dict_list)

[{'_id': ObjectId('5eb6bcab60e378f730f5b62f'),
  'conf_name': 'SSII2019',
  'content': {'序論': '近年，自動車の自動運転化が進んでおり，自動運転に\n'
                    '関する研究，開発が盛んに行われている．人の判断や\n'
                    '操作を必要としない完全な自動運転では，周辺環境を\n'
                    '認識し，事故リスク推定を行う必要がある. 認識の分野\n'
                    'では，深層学習を用いた検出手法 [1][2] 等にて，高い精\n'
                    '度が報告されている．一方，事故リスク推定としては，\n'
                    '手法 [3] 等が存在するが，実用に足る精度の手法が確立\n'
                    'しておらず，事故リスク推定手法の需要が高まってい\n'
                    'る．また、二輪車の事故が発生した際の致死率は，四輪\n'
                    '車の約 2 倍以上となっている [4]．しかし，二輪車に安\n'
                    '全運転支援の機能を搭載することは困難である．二輪\n'
                    '車は重心の移動によって自立, 旋回が可能であるため,\n'
                    '四輪車と異なり重心の制御も行う必要がある．ところ\n'
                    'が，重心は乗車している人の動作によって変化するため\n'
                    '汎用的な重心制御の技術が必要となる. しかしながら,\n'
                    '二輪車はその車体特性上, 多くの重心制御の部品 (バラ\n'
                    'ンサー) を搭載することは現実的ではない. そのため，\n'
                    '二輪車の安全運転支援機能の搭載は四輪車よりもコス\n'
              

                    'ノテーションが付与されているが，フレームごとに危IS1-24\n'
                    'SO1-24第25回画像センシングシンポジウム険度アノテーションを付与し，事故発生までの時刻を\n'
                    '考慮した学習などを今後行っていく．参考文献[1] Shaoqing Ren，Kaiming He，Ross '
                    'Girshick，Jian\n'
                    'Sun，“ Faster R-CNN: Towards Real-Time Object\n'
                    'Detection with Region Proposal Networks ”，Neu-\n'
                    'ral Information Processing Systems 2015，2015.[2] Wei Liu, '
                    'Dragomir Anguelov,Dumitru Er-han,Christian Szegedy,Scott '
                    'Reed, Cheng-Yang\n'
                    'Fu, Alexander C. Berg,“SSD:Single ShotMulti Box\n'
                    'Detector ”,European Conference on Computer\n'
                    'Vision 2016,pp21-37,2016.[3] Fu-Hsiang Chan, Yu-Ting '
                    'Chen, Yu Xiang, Min\n'
                    'Sun,“Anticipating Accidents in Dashcam Videos”,\n'
                    'ACCV2016, pp.136-153, 2016.[4] 警察庁交通局，“ 平成 29 '
                

                    '較的複雑な深層ニューラルネットの組み合わせで構成\n'
                    'されており，約 55 万フレームの成人注視マップデータ\n'
                    'を用いて学習されている．対して本稿の学習データは 1\n'
                    '万を下回る規模であり，(cid:12)ne-tuning する場合であって\n'
                    'も十分に高い汎化性能を獲得しにくかったことが原因Copyright © SSII 2019. All Rights '
                    'Reserved.- IS1-25 -として考えられる．上記の結果より，提案法の有用性\n'
                    'が確認できたと言える．\n'
                    '3.4 定性評価結果続いて，定性的な評価結果として，図 3 に提案法，[4]，\n'
                    '及び [10] の推定結果例を示す．提案法の推定結果が正\n'
                    '解の注視マップに近いことが確認できる．最初の例（1\n'
                    '行目）は，右折動作中のシーンであり，他の手法は広\n'
                    'く注視領域を推定する傾向にある．一方，高齢者の注\n'
                    '視は交差点を渡ろうとする歩行者付近に集中しており，\n'
                    '提案法はこの傾向を反映した推定結果となっている．ま\n'
                    'た，二番目，三番目の例では，左折しようとする前方\n'
                    'の車に注意が集中する傾向を反映した推定結果となっ\n'
                    'ている．[4] も比較的同様の推定結果を出しているもの\n'
                    'の，成人の傾向である注視領域が広がりやすい傾向を\n'
                    '反映したものになっており，提案法

  'content': {'序論': 'Copyright © SSII 2019. All Rights Reserved.- IS1-34 '
                    '-レードオフを考える必要がある．\n'
                    '本研究では，3D CNNによる動画認識をするうえで，\n'
                    '認識性能とコストのトレードオフを考えるための基礎\n'
                    '実験に取り組む．本研究では，特に認識精度と計算コス\n'
                    'ト (FLOPs)の関係についての実験をすることで，この\n'
                    'トレードオフについて議論する．これにより，3D CNN\n'
                    'を実際に用いる際の入力サイズの設定方法を明らかに\n'
                    'する．\n'
                    '2 実験設定\n'
                    '本研究では，著者らの先行研究で提案した 3D CNN\n'
                    'のネットワーク構造である 3D ResNet-18 [2]を利用し，\n'
                    '入力サイズが認識性能と計算コストに与える影響につ\n'
                    'いて実験的に議論する．\n'
                    '以下では，本実験の具体的な設定について述べる．\n'
                    '2.1 学習\n'
                    'ネットワークの学習は確率的勾配降下法 (SGD) に\n'
                    'よって行う．学習サンプルの生成は，学習データの動画\n'
                    '中からランダムに生成する形で Data Augmentationしな\n'
                    'がら行う．具体的には，まず各サンプルの時間位置を動\n'
                    '画中から一様分布に従ってランダム

## dictのリストのパース

メモリを削減するためdataloaderからfindしたリストそれぞれをPaperオブジェクトに変換する

### DocumentのリストからPaperのリストを取得

In [20]:
class DictListPdfParser():
    def __init__(self, pdf_parser):
        self.pdf_parser = pdf_parser

    def parse_from_dict_list(self, dict_list):
        paper_list = [self.pdf_parser.parse_by_dict(one_dict) for one_dict in dict_list]
        return paper_list

In [21]:
list_paper_parser = PdfParserCount(count_patterns=find_patterns)
dict_list_pdf_parser = DictListPdfParser(pdf_parser=list_paper_parser)

In [22]:
paper_list = dict_list_pdf_parser.parse_from_dict_list(dict_list)
paper_list.sort(key=lambda paper_counter: tuple(paper_counter.counters.values()),reverse=True)
print(paper_list)

[IS1-24
車載カメラ映像からの二輪車との交通事故リスク推定
OrderedDict([(re.compile('ディープラーニング|深層学習'), 6), (re.compile('CNN|ニューラルネットワーク'), 24), (re.compile('VAE|変分オートエンコーダ'), 0), (re.compile('GAN'), 0)])
2000-01-01 00:00:00, IS1-31

OrderedDict([(re.compile('ディープラーニング|深層学習'), 3), (re.compile('CNN|ニューラルネットワーク'), 6), (re.compile('VAE|変分オートエンコーダ'), 0), (re.compile('GAN'), 0)])
2000-01-01 00:00:00, IS1-26
深層学習による水中映像からのマグロ成魚の尾数推定
OrderedDict([(re.compile('ディープラーニング|深層学習'), 2), (re.compile('CNN|ニューラルネットワーク'), 2), (re.compile('VAE|変分オートエンコーダ'), 0), (re.compile('GAN'), 0)])
2000-01-01 00:00:00, IS1-41
カラー画像と距離情報を統合した深層学習による高精度な鉄塔抽出技術
OrderedDict([(re.compile('ディープラーニング|深層学習'), 1), (re.compile('CNN|ニューラルネットワーク'), 2), (re.compile('VAE|変分オートエンコーダ'), 0), (re.compile('GAN'), 0)])
2000-01-01 00:00:00, IS1-32
カメラ映像を用いた cGAN に基づく深度推定に関する検討
OrderedDict([(re.compile('ディープラーニング|深層学習'), 0), (re.compile('CNN|ニューラルネットワーク'), 15), (re.compile('VAE|変分オートエンコーダ'), 0), (re.compile('GAN'), 6)])
2000-01-01 00:00:00, IS1-27

OrderedDict([(r